Scraping for playing probability for the current Serie A matchday.

Data from http://fantacalcio.it

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv


In [ ]:
res = requests.get('https://www.fantacalcio.it/probabili-formazioni-serie-a')

In [ ]:
soup = BeautifulSoup(res.text)

groups = soup.findAll('ul', attrs = {'class' : 'player-list'})

probables = pd.DataFrame(columns = ['player', 'starter', 'percentage'])

for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    bars = groups[i].findAll('div', attrs = {'class' : 'progress-bar'})
    
    starter_string = groups[i]['class'][1]
    if(starter_string == 'starters'):
        starter = 1
    else:
        starter = 0
    
    for j in range(len(players)):
        player = players[j].contents[1].contents[0]
        perc = float(bars[j]['aria-valuenow'])
        
        row_df = pd.DataFrame([[player, starter, perc]], columns = probables.columns)
        
        probables = pd.concat([probables, row_df], ignore_index = True)
    
probables   

In [ ]:
probables = probables.set_index('player')

In [ ]:
groups = soup.findAll('ul', attrs = {'class' : 'ballot-list'})

ballots = pd.DataFrame(columns = ['player1', 'player2', 'percentage'])
for i in range (len(groups)):
    players = groups[i].findAll('a', attrs = {'class' : 'player-name'})
    percs = groups[i].findAll('strong', attrs = {'class' : 'percentage'})
    
    player1 = players[0].contents[1].contents[0]
    player2 = players[1].contents[1].contents[0]
    perc = float(percs[0].contents[0].replace('%', ''))
    
    row_df = pd.DataFrame([[player1, player2, perc]], columns = ballots.columns)
        
    ballots = pd.concat([ballots, row_df], ignore_index = True)
                                  
ballots

In [ ]:
for i in range(ballots.shape[0]):
    p1 = ballots['player1'][i]
    p2 = ballots['player2'][i]
    
    perc = float(ballots['percentage'][i])
    
    probables.at[p1, 'starter'] = perc/100
    probables.at[p2, 'starter'] = 1 - perc/100
    
probables

In [ ]:
probables.to_excel('mid_outputs/match_probable_players.xlsx')   